# Data Cleaning

Before i can perform analysis on any of the data i will need to clean all the raw .csv files. Some have been downloaded from HTML format, and almost all of them come from different sources, so they'll all need different cleaning steps.

In order to perform timeseries analysis on them properly i'll also need to ensure that they have the same intervals and are across the same period (i.e. they have the same start and end dates to the samples). To make the data work with the most limiting dataset, the sample interval will need to be monthly, with the first sample being April 2002 and the most recent sample being November 2022. I have created a custom function that i'll use to convert each clean dataframe to this format.

Once all the dataframes have been cleaned and have uniform length and intervals, i'll join them in to one dataset to perform my analysis.

----

## Import Libraries

In [1]:
# Basic packages.
import numpy as np
import pandas as pd
import calendar
import datetime

# Custom helper functions.
import sys
sys.path.append('../libraries')
import cleaning_helper_functions as prep

----

## Read Data

In [2]:
# Read CSV files to pandas dataframes.
df_south_raw = pd.read_csv('../data/raw/S_seaice_extent_daily_v3.0.csv')
df_north_raw = pd.read_csv('../data/raw/N_seaice_extent_daily_v3.0.csv')
df_antarctica_ice_raw = pd.read_csv('../data/raw/antarctica_ice_sheet.csv')
df_greenland_ice_raw = pd.read_csv('../data/raw/greeland_ice_sheet.csv')
df_sea_level_raw = pd.read_csv('../data/raw/global_mean_sea_level.csv')
df_sea_temp_raw = pd.read_csv('../data/raw/sea_temperature.csv')

----

## Sea Ice

### Initial Inspection - South Sea Ice

In [3]:
prep.basic_eda(df_south_raw)

SHAPE
Rows: 14546 	 Columns: 6

DATATYPES
Columns: 
Year            object
 Month          object
 Day            object
     Extent     object
    Missing     object
 Source Data    object
dtype: object
Index: integer

UNIQUE VALUES
Year               47
 Month             13
 Day               32
     Extent      9306
    Missing         5
 Source Data    14546
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 0
Percentage duplicate rows: 0.0%

-----



In [4]:
df_south_raw.head()

,Year,Month,Day,Extent,Missing,Source Data
0,YYYY,MM,DD,10^6 sq km,10^6 sq km,Source data product web sites: http://nsidc.o...
1,1978,10,26,17.624,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.26/nt_19...
2,1978,10,28,17.803,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.28/nt_19...
3,1978,10,30,17.670,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.30/nt_19...
4,1978,11,01,17.527,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.11.01/nt_19...


### Initial Inspection - North Sea Ice

In [5]:
prep.basic_eda(df_north_raw)

SHAPE
Rows: 14546 	 Columns: 6

DATATYPES
Columns: 
Year            object
 Month          object
 Day            object
     Extent     object
    Missing     object
 Source Data    object
dtype: object
Index: integer

UNIQUE VALUES
Year               47
 Month             13
 Day               32
     Extent      8079
    Missing         2
 Source Data    14546
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 0
Percentage duplicate rows: 0.0%

-----



In [6]:
df_north_raw.head()

,Year,Month,Day,Extent,Missing,Source Data
0,YYYY,MM,DD,10^6 sq km,10^6 sq km,Source data product web sites: http://nsidc.o...
1,1978,10,26,10.231,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.26/nt_19...
2,1978,10,28,10.420,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.28/nt_19...
3,1978,10,30,10.557,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.30/nt_19...
4,1978,11,01,10.670,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.11.01/nt_19...


Observations:
- All column headers except `Year` are formatted with trailing or leading whitespace characters.
- Entries are made every two days to begin with, but every day by the end of the dataset.
- No Null values or duplicate rows.
- No duplicate columns.
- First data entry is an extended description of the column.
- All columns are set as object datatypes.
- There are very few rows missing data in the `Missing` column, and it won't be required for my analysis.
- The `Source Data` column just returns a list of links, so isn't helpful in my analysis.
- Both datasets have the same features (size, datatypes, formatting etc.).

### Cleaning - Sea Ice

As these two datasets are taken from the same source, and have the same layout etc. i will be able to define one function that carries out all the cleaning steps, and then enact that function on both datasets to speed up the cleaning process.

I'll also combine the northern and southern hemisphere data in to a 'Total Sea Ice Extent' column to use for my analysis, as it will look at global mean sea level.

Cleaning Steps:
- Remove whitespace characters from column headers.
- Extract information from and remove first row.
- Concatenate `Year`, `Month` and `Day` to datetime `Date` column.
- Set `Date` column as the index.
- Drop the `Source Data`, `Missing`, `Year`, `Month` and `Day` columns.
- Change `Extent` columns to `float` datatype.
- Join the two datasets.
- Standardise the reading frequency by including the missing days earlier in the dataset.
- Interpolate any null values created by the previous step.

In [7]:
# Create function 
def clean_sea_ice(df):
    """
    Performs required cleaning steps on the sea ice dataframes.

    Inputs:
        - pandas Dataframe.
    """
    # Remove whitespace characters from column headers.
    for col in df.columns:
        new_header = col.strip()
        df.rename(columns={col:new_header}, inplace=True)

    # Remove first row.
    df.drop(index=df.index[0], axis=0, inplace=True)

    # Create Date column from Year, Month and Day and set as index.
    df['Date'] = pd.to_datetime(df[['Year','Month','Day']])
    df.set_index('Date', inplace=True)

    # Drop unrequired columns.
    df.drop(['Year','Month','Day','Missing','Source Data'], axis=1, inplace=True)

    # Set Extent datatype as float. 
    df[['Extent']] = df[['Extent']].astype(float)
    return df
    

In [8]:
# Input each dataframe in to the function.
df_south_raw = clean_sea_ice(df_south_raw)
df_north_raw = clean_sea_ice(df_north_raw)

In [9]:
# Join datasets.
df_sea_ice = df_south_raw.join(df_north_raw, lsuffix='_South', rsuffix='_North')

# Reindex to include missing dates and interpolate new values.
df_sea_ice = df_sea_ice.reindex(pd.date_range(df_sea_ice.index[0],df_sea_ice.index[-1]))
df_sea_ice[['Extent_South','Extent_North']] = df_sea_ice[['Extent_South','Extent_North']].interpolate()

In [10]:
# Resample timeseries to required length.
df_sea_ice_resampled = prep.resample_timeseries(df_sea_ice)

In [11]:
# Create total column and drop original columns.
df_sea_ice_resampled['global_sea_ice_extent'] = df_sea_ice_resampled['Extent_North'] + df_sea_ice_resampled['Extent_South']
df_sea_ice_resampled.drop(['Extent_North','Extent_South'],axis=1,inplace=True)

### Review - Sea Ice

In [12]:
prep.basic_eda(df_sea_ice_resampled)

SHAPE
Rows: 248 	 Columns: 1

DATATYPES
Columns: 
global_sea_ice_extent    float64
dtype: object
Index: datetime64

UNIQUE VALUES
global_sea_ice_extent    248
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 0
Percentage duplicate rows: 0.0%

-----



In [13]:
df_sea_ice_resampled.head()

,global_sea_ice_extent
2002-04-30,20.627267
2002-05-31,22.482161
2002-06-30,23.959167
2002-07-31,24.688871
2002-08-31,23.665677


----

## Ice Sheets

### Initial Inspection - Antarctica Ice Sheet

In [14]:
prep.basic_eda(df_antarctica_ice_raw)

SHAPE
Rows: 245 	 Columns: 1

DATATYPES
Columns: 
HDR Antarctica Mass    object
dtype: object
Index: integer

UNIQUE VALUES
HDR Antarctica Mass    238
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 8
Percentage duplicate rows: 3.27%

-----



In [15]:
df_antarctica_ice_raw.head()

,HDR Antarctica Mass
0,HDR
1,HDR Data from the GRACE and GRACE-FO JPL RL06....
2,HDR
3,HDR This file contains values that are anomali...
4,HDR auspices of the NASA MEaSUREs program. The...


### Inspection - Greenland Ice Sheet

In [16]:
prep.basic_eda(df_greenland_ice_raw)

SHAPE
Rows: 245 	 Columns: 1

DATATYPES
Columns: 
HDR Greenland Mass    object
dtype: object
Index: integer

UNIQUE VALUES
HDR Greenland Mass    238
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 8
Percentage duplicate rows: 3.27%

-----



In [17]:
df_greenland_ice_raw.head()

,HDR Greenland Mass
0,HDR
1,HDR Data from the GRACE and GRACE-FO JPL RL06....
2,HDR
3,HDR This file contains values that are anomali...
4,HDR auspices of the NASA MEaSUREs program. The...


Using the views above, as well as looking at the raw data file, the following observations can be made.

Observations:
- Several lines of comment text at the head of the file.
- All data in one column.
- No null values or duplicate rows or columns.
- Date is given as a decimal.
- `Mass_Gigatonnes_1_sigma_unc` will not be useful in my analysis.
- All columns are object datatype.

### Cleaning - Ice Sheets

As with the sea ice extent data, it looks like both datasets have the same layout and features. I'll therefore be able to define another function that i can feed both datasets through to clean them.

Again, i'll also combine both columns in to a total column to use in my analysis going forwards.

Cleaning Steps:
- Remove all text at the head of the file.
- Split the data in to multiple columns.
- Rename columns.
- Create `Date` column by converting `Date_Decimal` to datetime.
- Set `Date` as index.
- Drop `Mass_Gigatonnes_1_sigma_unc` and `Date_Decimal`.
- Convert `Mass_Gigatonnes` to float.
- Combine datasets.

In [18]:
def ice_sheet_cleaning(df):
    # Drop all text rows and reset index.
    df = df[df.iloc[:,0].str.startswith('HDR')==False].reset_index(drop=True)

    # Split into columns based on one or more whitespace characters as delimiters.
    df = df.iloc[:,0].str.split(' +',expand=True, regex=True) 

    # Rename columns.
    df = df.rename({0:'Decimal_Date',1:'Mass_Gigatonnes',2:'Mass_Gigatonnes_1_sigma_unc'},axis='columns')

    # Create date column from list of converted decimal dates and set as index.
    df['Date'] = pd.to_datetime([prep.decimal_to_datetime(date) for date in df['Decimal_Date'].astype(float).tolist()]) 
    df.set_index('Date', inplace=True)

    # Drop unrequired columns.
    df.drop(['Decimal_Date','Mass_Gigatonnes_1_sigma_unc'], axis=1, inplace=True)

    # Convert mass to float.
    df['Mass_Gigatonnes'] = df['Mass_Gigatonnes'].astype(float)
    return df

In [19]:
# Clean datasets.
df_greenland_ice_raw = ice_sheet_cleaning(df_greenland_ice_raw)
df_antarctica_ice_raw = ice_sheet_cleaning(df_antarctica_ice_raw)

# Join datasets.
df_ice_sheets = df_greenland_ice_raw.join(df_antarctica_ice_raw, lsuffix='_Greenland', rsuffix='_Antarctica')

# Reindex the dataframe to include the missing dates and interpolate new values.
df_ice_sheets = df_ice_sheets.reindex(pd.date_range(df_ice_sheets.index[0],df_ice_sheets.index[-1]))
df_ice_sheets[['Mass_Gigatonnes_Greenland','Mass_Gigatonnes_Antarctica']] = \
    df_ice_sheets[['Mass_Gigatonnes_Greenland','Mass_Gigatonnes_Antarctica']].interpolate()

In [20]:
# Resample timeseries to match required length.
df_ice_sheets_resampled = prep.resample_timeseries(df_ice_sheets)

In [21]:
df_ice_sheets_resampled['total_ice_sheet_mass_change'] = df_ice_sheets_resampled['Mass_Gigatonnes_Antarctica'] + df_ice_sheets_resampled['Mass_Gigatonnes_Greenland']
df_ice_sheets_resampled.drop(['Mass_Gigatonnes_Antarctica','Mass_Gigatonnes_Greenland'],axis=1,inplace=True)

### Review - Ice Sheets

In [22]:
prep.basic_eda(df_ice_sheets_resampled)

SHAPE
Rows: 248 	 Columns: 1

DATATYPES
Columns: 
total_ice_sheet_mass_change    float64
dtype: object
Index: datetime64

UNIQUE VALUES
total_ice_sheet_mass_change    248
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 0
Percentage duplicate rows: 0.0%

-----



In [23]:
df_ice_sheets_resampled.head()

,total_ice_sheet_mass_change
2002-04-30,27.303182
2002-05-31,49.320792
2002-06-30,-56.745000
2002-07-31,-169.678182
2002-08-31,-257.390352


----

## Global Mean Sea Level

### Initial Inspection - Global Mean Sea Level

In [24]:
prep.basic_eda(df_sea_level_raw)

SHAPE
Rows: 1152 	 Columns: 1

DATATYPES
Columns: 
HDR Global Mean Sea Level Data    object
dtype: object
Index: integer

UNIQUE VALUES
HDR Global Mean Sea Level Data    1143
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 11
Percentage duplicate rows: 0.95%

-----



In [25]:
df_sea_level_raw.head()

,HDR Global Mean Sea Level Data
0,HDR
1,HDR This file contains Global Mean Sea Level (...
2,HDR auspices of the NASA Sea Level Change prog...
3,HDR Climate Research (http://podaac.jpl.nasa.g...
4,"HDR TOPEX/Poseidon, Jason-1, OSTM/Jason-2, Jas..."


Observations
- Readings are taken at non-uniform intervals.
- No clear date that readings are relative to.
- There is a lot of unnecessary data in the dataset. I will only want to keep `GMSL` and `year+fraction_of_year`.
- Date is given as a decimal.
- `GMSL` is currently an object datatype.

### Cleaning - Global Mean Sea Level

Initial Cleaning Steps:
- Remove all text at the head of the file.
- Split the data in to multiple columns.
- Rename columns.
- Drop unnecessary columns.
- Convert `year+fraction_of_year` to datetime `Date` column.
- Set `Date` as index.
- Convert `GMSL` to float.
- Expand index to include all missing dates.
- Interpolate data to fill added dates.
- Set `GMSL` for first entry to 0 and all other entries relative to it.

In [26]:
# Drop all text rows and reset index.
df_sea_level_raw = df_sea_level_raw[df_sea_level_raw.iloc[:,0].str.startswith('HDR')==False].reset_index(drop=True)

# Rename header.
df_sea_level_raw.columns = ['0'] 

# Append space to allow for easier delimitation and split columns.
df_sea_level_raw['0'] = ' ' + df_sea_level_raw['0'] 
df_sea_level_raw = df_sea_level_raw.iloc[:,0].str.split(' +',expand=True, regex=True)

# Remove first column and rename remaining columns.
df_sea_level_raw = df_sea_level_raw.iloc[:,1:] 
df_sea_level_raw = df_sea_level_raw.rename({1:'altimeter_type',
                                            2:'merged_file_cycle',
                                            3:'year+fraction_of_year',
                                            4:'no_observations',
                                            5:'no_weighted_obs',
                                            6:'GMSL',
                                            7:'STD_DEV',
                                            8:'smoothed_GMSL',
                                            9:'GMSL_variatiion',
                                            10:'std_dev_GMSL_variation',
                                            11:'smoothed_GMSL_variation',
                                            12:'smoothed_GMSL_variation_signal_rem'},axis='columns')

# Convert dates to caldendar date and set as index.
df_sea_level_raw['Date'] = \
    pd.to_datetime([prep.decimal_to_datetime(date) for date in df_sea_level_raw['year+fraction_of_year'].astype(float).tolist()])
df_sea_level_raw.set_index('Date', inplace=True)

# Filter GMSL and set as float.
df_sea_level_raw = df_sea_level_raw[['GMSL']]
df_sea_level_raw[['GMSL']] = df_sea_level_raw[['GMSL']].astype(float)

# Reindex the dataframe to include the missing dates and interpolate new values.
df_sea_level_raw = df_sea_level_raw.reindex(pd.date_range(df_sea_level_raw.index[0],df_sea_level_raw.index[-1]))
df_sea_level_raw['GMSL'] = df_sea_level_raw['GMSL'].interpolate()

In [27]:
# Resample timeseries to match required length.
df_sea_level_resampled = prep.resample_timeseries(df_sea_level_raw)

# Adjust GMSL to be relative to start of sampling period.
df_sea_level_resampled['GMSL'] = round(df_sea_level_resampled['GMSL'] + abs(df_sea_level_resampled['GMSL'][0]),2)

### Review - Global Mean Sea Level

In [28]:
prep.basic_eda(df_sea_level_resampled)

SHAPE
Rows: 248 	 Columns: 1

DATATYPES
Columns: 
GMSL    float64
dtype: object
Index: datetime64

UNIQUE VALUES
GMSL    246
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 4
Percentage duplicate rows: 1.61%

-----



In [29]:
df_sea_level_resampled.head()

,GMSL
2002-04-30,0.00
2002-05-31,-1.34
2002-06-30,-1.69
2002-07-31,3.42
2002-08-31,8.99


----

## Sea Temperature

### Initial Inspection - Sea Temperature

In [30]:
prep.basic_eda(df_sea_temp_raw)

SHAPE
Rows: 2077 	 Columns: 10

DATATYPES
Columns: 
year                                          int64
month                                         int64
anomaly                                     float64
total_uncertainty                           float64
uncorrelated_uncertainty                    float64
correlated_uncertainty                      float64
bias_uncertainty                            float64
coverage_uncertainty                        float64
lower_bound_95pct_bias_uncertainty_range    float64
upper_bound_95pct_bias_uncertainty_range    float64
dtype: object
Index: integer

UNIQUE VALUES
year                                         174
month                                         12
anomaly                                     2051
total_uncertainty                           1796
uncorrelated_uncertainty                    1226
correlated_uncertainty                      1691
bias_uncertainty                            1409
coverage_uncertainty                     

In [31]:
df_sea_temp_raw.head()

,year,month,anomaly,total_uncertainty,uncorrelated_uncertainty,correlated_uncertainty,bias_uncertainty,coverage_uncertainty,lower_bound_95pct_bias_uncertainty_range,upper_bound_95pct_bias_uncertainty_range
0,1850,1,-0.35138,0.14007,0.03096,0.09622,0.04578,0.08547,-0.4530,-0.2695
1,1850,2,-0.34437,0.13852,0.03158,0.09198,0.04642,0.08703,-0.4475,-0.2615
2,1850,3,-0.58001,0.15060,0.03537,0.09765,0.04425,0.09968,-0.6780,-0.5005
3,1850,4,-0.34222,0.13522,0.03352,0.09939,0.04384,0.07322,-0.4390,-0.2630
4,1850,5,-0.25093,0.12226,0.03314,0.09200,0.03897,0.06218,-0.3375,-0.1810


Observations:
- There are a lot of different metrics given. I'll only need the date and the anomaly data for my analysis.

### Cleaning - Sea Temperature

Cleaning Steps:
- Remove unnecessary columns.
- Create date column and set as index.

In [32]:
# Extract useful columns.
df_sea_temp_raw = df_sea_temp_raw[['year','month','anomaly']]

# Create date column and set as index.
df_sea_temp_raw['Date'] = \
    pd.to_datetime(dict(year=df_sea_temp_raw['year'],month=df_sea_temp_raw['month'], day=1)) + pd.offsets.MonthEnd()
df_sea_temp_raw.drop(['year','month'],axis=1,inplace=True)
df_sea_temp_raw.set_index('Date',inplace=True)

In [33]:
# Resample timeseries to match required length.
df_sea_temp_resampled = prep.resample_timeseries(df_sea_temp_raw)

### Review - Sea Temperature

In [34]:
prep.basic_eda(df_sea_temp_resampled)

SHAPE
Rows: 248 	 Columns: 1

DATATYPES
Columns: 
anomaly    float64
dtype: object
Index: datetime64

UNIQUE VALUES
anomaly    247
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 2
Percentage duplicate rows: 0.81%

-----



In [35]:
df_sea_temp_resampled.head()

,anomaly
Date,
2002-04-30,0.45826
2002-05-31,0.46104
2002-06-30,0.46607
2002-07-31,0.41203
2002-08-31,0.43217


----

## Combine DataFrames

In [36]:
# Check shape of all data frames.
frames = [df_sea_ice_resampled,df_sea_temp_resampled,df_sea_level_resampled,df_ice_sheets_resampled]

for frame in frames:
    print(frame.shape)

(248, 1)
(248, 1)
(248, 1)
(248, 1)


In [37]:
# Concatenate dataframes.
earth_vitals_df = pd.concat([df_sea_ice_resampled, df_sea_temp_resampled, \
                             df_ice_sheets_resampled,df_sea_level_resampled], axis=1)

# Rename columns.
earth_vitals_df.rename(columns={'global_sea_ice_extent':'global_sea_ice_extent',
                                'anomaly':'sea_temp_anomaly',
                                'total_ice_sheet_mass_loss':'total_ice_sheet_mass_loss',
                                'GMSL':'global_mean_sea_level'
                                }, inplace=True)

In [38]:
prep.basic_eda(earth_vitals_df)

SHAPE
Rows: 248 	 Columns: 4

DATATYPES
Columns: 
global_sea_ice_extent          float64
sea_temp_anomaly               float64
total_ice_sheet_mass_change    float64
global_mean_sea_level          float64
dtype: object
Index: datetime64

UNIQUE VALUES
global_sea_ice_extent          248
sea_temp_anomaly               247
total_ice_sheet_mass_change    248
global_mean_sea_level          246
dtype: int64

NULL VALUES
Total null rows: 0
Percentage null rows: 0.0%

DUPLICATES
Total duplicate rows: 0
Percentage duplicate rows: 0.0%

-----



In [39]:
earth_vitals_df.head()

,global_sea_ice_extent,sea_temp_anomaly,total_ice_sheet_mass_change,global_mean_sea_level
2002-04-30,20.627267,0.45826,27.303182,0.00
2002-05-31,22.482161,0.46104,49.320792,-1.34
2002-06-30,23.959167,0.46607,-56.745000,-1.69
2002-07-31,24.688871,0.41203,-169.678182,3.42
2002-08-31,23.665677,0.43217,-257.390352,8.99


----

### Export Clean CSV

In [40]:
earth_vitals_df.to_csv('../data/cleaned/earth_vitals.csv')

----